In [ ]:
%run "Setup_DesignStart.ipynb"

In [ ]:
print(target.get_target_name())
print(target.get_fw_buildtime())
print(target.get_fpga_buildtime())

In [ ]:
# stable trigger:
target.fpga_write(target.REG_COUNTER_QUICK_START, [1])
target.fpga_write(target.REG_CAPTURE_RAW, [0])
target.fpga_write(target.REG_COUNT_WRITES, [0])
target.fpga_write(target.REG_SOFT_TRIG_ENABLE, [1])
target.fpga_write(target.REG_SOFT_TRIG_PASSTHRU, [1])
target.fpga_write(target.REG_PATTERN_TRIG_ENABLE, [0])

In [ ]:
# match on any PC trace packet:
target.set_pattern_match(0, [5, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])
#target.set_pattern_match(0, [5, 8, 32, 229, 2, 0, 0, 132], [255, 255, 255, 255, 255, 255, 255, 0])

# enable matching rule:
target.fpga_write(target.REG_PATTERN_ENABLE, [1])

target.fpga_write(target.REG_CAPTURE_LEN, int.to_bytes(10000, length=4, byteorder='little'))

In [ ]:
project = cw.create_project("projects/jupyter_tvla_sw.cwp", overwrite=True)

In [ ]:
# Capture Traces for TVLA test - no tracing here, we first need to find where the TVLA failures are
import chipwhisperer as cw
from tqdm import tnrange
import numpy as np
import time

N = 50  # Number of traces

ktp = cw.ktp.TVLATTest()
ktp.init(N)
traces = []

keys = []
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()  # TVLA T-Test changes PT between two options
    keys.append(key)
    trace = cw.capture_trace(scope, target.ss, text, key)

    if trace is None:
        continue
    project.traces.append(trace)

In [ ]:
project.save()

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import scipy
import scipy.stats

project = cw.open_project('projects/jupyter_tvla_sw.cwp')
fixedpy = [0xda, 0x39, 0xa3, 0xee, 0x5e, 0x6b, 0x4b, 0x0d, 0x32, 0x55, 0xbf, 0xef, 0x95, 0x60, 0x18, 0x90]

testouts = []
num_traces = len(project.traces)
num_points = len(project.waves[0])
print(num_traces, num_points)
curve = hv.Curve([])

def do_the_ttvla(project, ntraces=-1):
    global curve, line
    if ntraces == -1:
        ntraces = int(num_traces / 2)
        
    if ntraces * 2 > num_traces:
        raise ValueError("Invalid ntraces")
    for g in range(0, 2):
        group = [(list(project.textins[i]) == fixedpy) for i in range(g*ntraces, g*ntraces+ntraces)]
        trace = np.zeros((ntraces, num_points))

        for n in range(g*ntraces, g*ntraces+ntraces):
            trace[n - g*ntraces][:] = project.waves[n]
                
        testout = welch_ttest(group, trace)
        curve *= hv.Curve(testout)
        testouts.extend(testout)
        
    curve *= hv.Path([(0, -4.5), (num_points, -4.5)]).opts(color="black") * \
    hv.Path([(0, 4.5), (num_points, 4.5)]).opts(color="black")

def welch_ttest(group, traces):
    import warnings
    # Compute Welch's t-statistic at each point in time
    # Here, group[] must only contain booleans (True/False)
    traces_true = traces[np.where(np.array(group))]
    traces_false = traces[np.where(~np.array(group))]
    
    if len(traces_true) == 0:
        traces_true  = np.array([[np.nan for _ in range(len(traces[0]))]])
    if len(traces_false) == 0:
        traces_false = np.array([[np.nan for _ in range(len(traces[0]))]])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ttrace = scipy.stats.ttest_ind(traces_true, traces_false, axis=0, equal_var=False)[0]
        
    return np.nan_to_num(ttrace) 
    
do_the_ttvla(project)

In [ ]:
len(np.where(abs(np.asarray(testouts)) > 4.5)[0])

In [ ]:
for t in times:
    vline = hv.VLine(t[0]*multiplier).opts(color='black', width=1)
    curve *= vline

In [ ]:
(curve).opts(height=600, width=1000)

In [ ]:
sort_indices = np.argsort(abs(np.asarray(testouts)))

In [ ]:
testouts[22203]

In [ ]:
sort_indices[-10:] -20000

# Now manually find PC where TVLA fails badly:

In [ ]:
#target.set_reg('ETM_TEEVR', '000150a0') # COMP0 or COMP1
target.set_reg('ETM_TEEVR', '00000020') # COMP0 only
#target.set_reg('ETM_TEEVR', '00000021') # COMP1 only

In [ ]:
target.set_reg('DWT_COMP0', '000012fc')
target.set_reg('DWT_COMP1', '00001394')

In [ ]:
# LOOKING FOR 1260!
#target.set_reg('DWT_COMP0', '000012c4') # 1266
#target.set_reg('DWT_COMP0', '000012c8') # 2236, 2284
#target.set_reg('DWT_COMP0', '000012c2') # 1185, 2199
#target.set_reg('DWT_COMP0', '000012c4') ## 1404, 2844
#target.set_reg('DWT_COMP0', '000012f8') # 1900, 2408
#target.set_reg('DWT_COMP0', '000012d0') # 1880, 2388
#target.set_reg('DWT_COMP0', '00001208') ## 2276
#target.set_reg('DWT_COMP0', '0000123c') # 2124, 2632
#target.set_reg('DWT_COMP0', '0000128e') # 2176, 2684
#target.set_reg('DWT_COMP0', '0000139a') # 2180
#target.set_reg('DWT_COMP0', '0000138c') ## 2244
#target.set_reg('DWT_COMP0', '00001358') ## 2192
#target.set_reg('DWT_COMP0', '00001376') ## 2220
#target.set_reg('DWT_COMP0', '00001370') ## 2212
target.set_reg('DWT_COMP0', '00001366') ## 2208

#target.set_reg('DWT_COMP0', '000012a2') # 2186
#target.set_reg('DWT_COMP0', '000012a6') # 

In [ ]:
#target.arm_trace()
#target.synced()
#target.simpleserial_write('p', bytearray(16), printresult=True)

In [ ]:
target.arm_trace()
trace = cw.capture_trace(scope, target.ss, text, key)

In [ ]:
target.fpga_read(target.REG_TRACE_COUNT, 8)

In [ ]:
target.fifo_empty()

In [ ]:
raw = target.read_capture_data()

In [ ]:
len(raw)

In [ ]:
#target.print_raw_data(raw)

In [ ]:
times = target.get_rule_match_times(raw, rawtimes=False, verbose=False)

In [ ]:
#times = target.get_rule_match_times(raw, rawtimes=True, verbose=True)

In [ ]:
len(times)

In [ ]:
for t in times:
    print(t[0]*multiplier)

In [ ]:
times

In [ ]:
# plot!

In [ ]:
if scope.clock.adc_src == 'clkgen_x4' or scope.clock.adc_src == 'extclk_x4':
    multiplier = 4
else:
    multiplier = 1

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.models import Span

output_notebook(INLINE)
p = figure(plot_width=1500)

xrange = range(len(trace.wave))
p.line(xrange, trace.wave, line_color="red")

vlines = []
for t in times:
    vlines.append(Span(location=t[0]*multiplier, dimension='height', line_color='black', line_width=2))
p.renderers.extend(vlines)

In [ ]:
show(p)